In [ ]:
import os

# competition_folder = '/kaggle/input/cafa-6-protein-function-prediction/'
competition_folder = 'data'

list_of_filenames = []
for dirname, _, filenames in os.walk(competition_folder):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        list_of_filenames.append(filepath)
        print(filepath)

-------
-------
-------
# Install Necessary Libraries

In [ ]:
# !pip install Bio fair-esm goatools propy3

-------
-------
-------
# Imports

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

from Bio import SeqIO
from goatools.obo_parser import GODag
from propy import PyPro
from propy.GetProteinFromUniprot import GetProteinSequence
import esm
import torch

-------
-------
-------
# Observe File Contents (Terminal)
First, let's have a look at the first few lines of each file to know what we are dealing with.

In [ ]:
train_sequences_file = competition_folder + '/Train/train_sequences.fasta'
!head {train_sequences_file} -n 10

In [ ]:
train_terms_file = competition_folder + '/Train/train_terms.tsv'
!head {train_terms_file} -n 10

In [ ]:
train_taxonomy_file = competition_folder + '/Train/train_taxonomy.tsv'
!head {train_taxonomy_file} -n 10

In [ ]:
go_file = competition_folder + '/Train/go-basic.obo'
!head {go_file} -n 55

In [ ]:
testsuperset_file = competition_folder + '/Test/testsuperset.fasta'
!head {testsuperset_file} -n 20

In [ ]:
testsuperset_taxon_file = competition_folder + '/Test/testsuperset-taxon-list.tsv'
!head {testsuperset_taxon_file} -n 10

In [ ]:
ia_file = competition_folder + '/IA.tsv'
!head {ia_file} -n 10

In [ ]:
sample_submission_file = competition_folder + '/sample_submission.tsv'
!head {sample_submission_file} -n 20

-------
-------
-------
# EDA
We shall load the different files, get some stats on the data, and observe some examples.

## Species Lookup
The spiecies IDs and their corresponding names.

In [ ]:
# species lookup table

species_lookup_df = pd.read_csv(testsuperset_taxon_file, sep="\t")
species_lookup_df.columns = ['species_id', 'species_name']
species_lookup_df

## FASTA Sequences

In [ ]:
# load the files containing the FASTA sequences

train_fasta_sequences = [{"id": record.id, "seq": record.seq} for record in SeqIO.parse(train_sequences_file, "fasta")]
test_fasta_sequences  = [{"id": record.id, "seq": record.seq} for record in SeqIO.parse(testsuperset_file,    "fasta")]

In [ ]:
# number of FASTA sequences in train & test sets

print(f"len(train_fasta_sequences) = {len(train_fasta_sequences)}")
print(f"len(test_fasta_sequences)  = {len(test_fasta_sequences)}")

In [ ]:
# look at data sample (first 3 examples of train set)

for record in train_fasta_sequences[:3]:
    print(record['id'])
    print(record['seq'])
    print('')

In [ ]:
lengths = [len(str(entry["seq"])) for entry in train_fasta_sequences]

bin_width = 100
bins = np.arange(0, max(lengths) + bin_width, bin_width)

plt.figure()
plt.hist(lengths, bins = bins)
plt.xlabel("Sequence Length")
plt.ylabel("Count")
plt.title("Distribution of FASTA Sequence Lengths (TRAIN)")
plt.show()

The majority of the sequences are below 5K amino acids long. There are a few very long sequences as well (> 35K amino acids).

Let us have a look at some numeric stats as well as the 99th percentile.

In [ ]:
print(pd.Series(lengths).describe())
print('')
print('')
p99 = np.percentile(lengths, 99)
print("99th percentile sequence length:", p99)

Some of the proteins are really tiny (< 10 amino acids). Guess in that case, they should be called peptides, not proteins. :)

Furthermore, 3/4 of all proteins are <= 526 amino acids long (75th percentile).

Now let's do the same but for the test set!

In [ ]:
lengths = [len(str(entry["seq"])) for entry in test_fasta_sequences]

print(pd.Series(lengths).value_counts().sort_index())
print('')
print('')
p99 = np.percentile(lengths, 99)
print("99th percentile sequence length:", p99)

bin_width = 100
bins = np.arange(0, max(lengths) + bin_width, bin_width)
plt.figure()
plt.hist(lengths, bins = bins)
plt.xlabel("Sequence Length")
plt.ylabel("Count")
plt.title("Distribution of FASTA Sequence Lengths (TEST)")
plt.show()

## Training Set GO Terms

In [ ]:
# load training set terms

train_terms_df = pd.read_csv(train_terms_file, sep="\t")
train_terms_df

In [ ]:
# number of unique values in each column

print(f"Number of unique entries: {train_terms_df.EntryID.nunique()}")
print(f"Number of unique terms:   {train_terms_df.term.nunique()}")
print(f"Number of unique aspects: {train_terms_df.aspect.nunique()}")

**From the cell above, It seems that what we have here is a *"multi-label classification problem"* involving 26,125 labels!**

That is, for each of the FASTA sequences, we are trying to predict which of the labels apply to it.

In [ ]:
# total number of terms + number of unique terms per aspect

print('Total number of terms:', len(train_terms_df))
print('')
print(train_terms_df['aspect'].value_counts())

In [ ]:
# NO intersections between terms of different aspects (as expected)

p_terms = train_terms_df[train_terms_df['aspect'] == "P"].term
c_terms = train_terms_df[train_terms_df['aspect'] == "C"].term
f_terms = train_terms_df[train_terms_df['aspect'] == "F"].term

print(set(p_terms).intersection(set(c_terms)))  #
print(set(p_terms).intersection(set(f_terms)))  # empty sets
print(set(c_terms).intersection(set(f_terms)))  #

In [ ]:
# number of terms per training sequence on average  ==>  ~6.5

train_terms_df.value_counts('EntryID').describe()

## Training Set Taxonomies (Species)

In [ ]:
# species id for each fasta record in the train set

train_species_df = pd.read_csv(train_taxonomy_file, sep="\t", header=None)
train_species_df.columns = ['fasta_id', 'species_id']
train_species_df

In [ ]:
train_species_df = pd.merge(train_species_df, species_lookup_df, on='species_id', how='inner')
train_species_df[['species_id', 'species_name']].value_counts()

## Gene Ontology DAG

In [ ]:
ontology = GODag(go_file)

# Example: Look up a GO term
term = ontology["GO:0006915"]
print('term.name:', term.name)
print('term.namespace:', term.namespace)
# print(term.parents)
# print(term.children)
# print(term.level)
# print(term.depth)
# print(term.is_obsolete)
# print(term.alt_ids)
# print(term.is_a)

# for documentation for the GoTerm class, refer to:
# https://github.com/tanghaibao/goatools/blob/main/goatools/obo_parser.py#L157

In [ ]:
# number of terms in the GO graph

len(ontology)

In [ ]:
# keep elements of GO graph that also exist in training set

list_of_train_terms = list(train_terms_df['term'])
ontology_filtered = {k: ontology[k] for k in list_of_train_terms}
len(ontology_filtered)

In [ ]:
# look at an example GO term and its parents

random_go_term = 'GO:0043371'
random_go_term_parents = ontology_filtered[random_go_term].get_all_parents()

print(ontology_filtered[random_go_term])
print('')
print('Parents:')
for parent in random_go_term_parents:
    if parent in ontology_filtered.keys():
        print(ontology[parent])

-------
-------
-------
# Feature Engineering
We'll focus only on the training set for now.

## propy3

In [ ]:
# function for generating features from a protein sequence

def get_protein_descriptors(sequence):
    DesObj = PyPro.GetProDes(sequence)

    all_descriptors = {}
    all_descriptors = all_descriptors | DesObj.GetAAComp()     # Amino acid compositon descriptors (20)
    all_descriptors = all_descriptors | DesObj.GetDPComp()     # Dipeptide composition descriptors (400)
    all_descriptors = all_descriptors | DesObj.GetCTD()        # Composition Transition Distribution descriptors (147)
    # all_descriptors = all_descriptors | DesObj.GetGearyAuto()  # Geary autocorrelation descriptors (240)
    # all_descriptors = all_descriptors | DesObj.GetMoranAuto()  # Moran autocorrelation descriptors (240)
    # all_descriptors = all_descriptors | DesObj.GetQSO()        # Quasi sequence order descriptors (default is 50)
    # all_descriptors = all_descriptors | DesObj.GetSOCN()       # Sequence order coupling numbers (default is 45)

    return all_descriptors

In [ ]:
# example

get_protein_descriptors(train_fasta_sequences[0]['seq'])

In [ ]:
train_features = {}
for i in range(len(train_fasta_sequences)):
    key_i = train_fasta_sequences[i]['id']
    seq_i = train_fasta_sequences[i]['seq']
    train_features[key_i] = get_protein_descriptors(seq_i)
    if i % 10000 == 0:
        print(i, '-->', datetime.now())

train_features

## ESM Embeddings

In [ ]:
# generate ESM embeddings for the sequences ()

torch.set_float32_matmul_precision("medium")  # FASTER!

# Load model
#   For more models, check the "Available Models and Datasets" section at this link:
#     https://pypi.org/project/fair-esm/#available-models
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()

if torch.cuda.is_available():
    model.cuda()  # GPU support

# number of layers
LAYER = len(model.layers)

def embed_sequence(seq):
    batch = [("protein1", seq)]
    _, _, tokens = batch_converter(batch)
    with torch.no_grad():
        if torch.cuda.is_available():
            tokens = tokens.cuda()
        results = model(tokens, repr_layers=[LAYER])
    token_representations = results["representations"][LAYER]
    embedding = token_representations[0, 1:-1].mean(0)  # Mean pooling (skip CLS and EOS tokens)

    if torch.cuda.is_available():
        embedding = embedding.cpu()
        
    return embedding.numpy()

In [ ]:
# set aside very long sequences that may cause out-of-memory problems

max_len = 8000
train_long_sequences = [x for x in train_fasta_sequences if len(str(x["seq"])) > max_len]
test_long_sequences  = [x for x in  test_fasta_sequences if len(str(x["seq"])) > max_len]
len(train_long_sequences), len(test_long_sequences)

In [ ]:
# sequences that we will generate embeddings for

train_ok_sequences = [x for x in train_fasta_sequences if len(str(x["seq"])) <= max_len]
test_ok_sequences  = [x for x in  test_fasta_sequences if len(str(x["seq"])) <= max_len]
len(train_ok_sequences), len(test_ok_sequences)

In [ ]:
train_features2 = {}
for i in range(len(train_ok_sequences)):
    key_i = train_ok_sequences[i]['id']
    seq_i = train_ok_sequences[i]['seq']
    train_features2[key_i] = embed_sequence(seq_i)
    if i % 10000 == 0:
        print(i, '-->', datetime.now())

In [ ]:
# concatenate train_features & train_features2



In [ ]:
# # len(train_features2)
# lengths = [len(str(entry["seq"])) for entry in test_fasta_sequences]
# # print(max([x for x in lengths if 5000<x<8000]))
# print(len([x for x in lengths if x > 7000]))
# # print(sorted([x for x in lengths if x > 10000]))
# # tmp123 = [x['seq'] for x in test_fasta_sequences if len(x['seq']) == 7968][0]
# # embed_sequence(tmp123)

In [ ]:
# from sklearn.decomposition import PCA
# import numpy as np

# # Suppose features is a list of vectors from multiple proteins
# # e.g., features = [embed_sequence(seq) for seq in sequences.values()]
# features = np.array(features)

# pca = PCA(n_components=50)   # choose output dimension
# reduced = pca.fit_transform(features)

# print("Original shape:", features.shape)
# print("Reduced shape:", reduced.shape)
